In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
r = requests.get('https://www.themalaysianinsight.com')

In [3]:
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]

In [5]:
topics = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/c/' in a_]))
topics

['https://www.themalaysianinsight.com/c/voices',
 'https://www.themalaysianinsight.com/c/pictures',
 'https://www.themalaysianinsight.com/c/other-news',
 'https://www.themalaysianinsight.com/c/editorial',
 'https://www.themalaysianinsight.com/c/videos',
 'https://www.themalaysianinsight.com/c/most-read',
 'https://www.themalaysianinsight.com/c/your-insight',
 'https://www.themalaysianinsight.com/c/notices',
 'https://www.themalaysianinsight.com/c/news']

In [7]:
topic_c = '-'.join(topics[0].split('/')[-2:])

'c-voices'

In [8]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    divs = soup.find_all('div', {'class': 'story'})
    urls = []
    for div in divs:
        a = div.find_all('a')
        a = [a_.get('href') for a_ in a if a_.get('href')]
        a = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/' in a_]))
        urls.extend(a)
    with open(filename, 'w') as fopen:
        json.dump(urls, fopen)
    return len(urls) > 0

In [9]:
directory = 'url'
!mkdir {directory}

mkdir: cannot create directory ‘url’: File exists


In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for topic in topics:
    if 'most-read' in topic:
        continue
    topic_c = '-'.join(topic.split('/')[-2:])
    print(topic_c)
    done = False
    for i in tqdm(range(0, 5000, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{topic}?page={a}', 
                 os.path.join(directory, f'{topic_c}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
        
        
        for future in as_completed(futures):
            if not future.result():
                done = True
        
        if done:
            break

c-voices


  2%|▋                                          | 8/500 [00:08<08:31,  1.04s/it]


c-pictures


  1%|▌                                          | 7/500 [00:00<00:16, 29.22it/s]


c-other-news


...

  3%|█▍                                        | 17/500 [00:07<03:23,  2.37it/s]


c-notices


  0%|                                           | 1/500 [00:07<58:37,  7.05s/it]


c-news


 69%|████████████████████████████▎            | 346/500 [51:15<22:48,  8.89s/it]
